In [5]:
%load_ext autoreload
%autoreload 2

In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt

2024-03-24 19:03:55.518426: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [51]:
train_file = "enemyHead_train.tfrecord"
train_dataset = tf.data.TFRecordDataset(train_file)

valid_file = "enemyHead_valid.tfrecord"
valid_dataset = tf.data.TFRecordDataset(valid_file)

In [52]:
# for raw_record in dataset.take(10):
#     print(repr(raw_record))
    

In [68]:
from functools import partial

keys_to_features = {
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'image/object/class/text': tf.io.VarLenFeature(tf.string),
    'image/object/class/label': tf.io.VarLenFeature(tf.int64),
}

def _parse_function(proto, returnText=False):
    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    
    # Decode the image
    image = tf.image.decode_jpeg(parsed_features['image/encoded'], channels=3)
    image = tf.image.resize(image, [416, 416])
    image = image / 255.0  # Normalize pixel values
    
    # Convert sparse tensor to dense, handling multiple entries if necessary
    label = tf.sparse.to_dense(parsed_features['image/object/class/label'])
    text = tf.sparse.to_dense(parsed_features['image/object/class/text'])

    # Decode bytes to string for text
    text = tf.cast(text, tf.string)
    
    # Check if the label contains '2', and set label to True in that case
    contains_label_2 = tf.reduce_any(tf.equal(label, 2))
    label = tf.cast(contains_label_2, tf.float32)  # Convert boolean to float (1.0 for True, 0.0 for False)

    if returnText:
        return image, label, text
    return image, label

def save_images(dataset):
    parseFunction = partial(_parse_function, returnText=True)
    # Map the dataset to parse the features
    images = dataset.map(parseFunction)
    
    # Directory to save the extracted images
    output_dir = 'extracted_images'
    os.makedirs(output_dir, exist_ok=True)
                
    # Iterate over each example
    for i, (image_tensor, label_tensor, text_tensor) in enumerate(images):
        # Save the image
        image_path = os.path.join(output_dir, f'image_{i}.jpg')
        # tf.io.write_file(image_path, tf.image.encode_jpeg(image_tensor))
        print(f'Saved {image_path}')
        print(f'Label: {label_tensor.numpy()}, Text: {text_tensor.numpy()}')
save_images(valid_dataset)

Saved extracted_images/image_0.jpg
Label: 1.0, Text: [b'enemyHead']
Saved extracted_images/image_1.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_2.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_3.jpg
Label: 1.0, Text: [b'enemyHead']
Saved extracted_images/image_4.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_5.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_6.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_7.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_8.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_9.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_10.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_11.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_12.jpg
Label: 1.0, Text

2024-03-24 19:32:54.136083: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [69]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(416, 416, 3)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 416, 416, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 208, 208, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 2768896)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 2768896)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │     2,768,897 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,770,689 (10.57 MB)

 Trainable params: 2,770,689 (10.57 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
batched_train = train_dataset.map(_parse_function).shuffle(1024).batch(32)
batched_valid = valid_dataset.map(_parse_function).shuffle(1024).batch(32)

history = model.fit(batched_train, epochs=10)

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.8247 - loss: 10.3294
Epoch 2/10


2024-03-24 19:34:33.437892: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/max/anaconda3/envs/CS6476Project/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


14/14 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.8636 - loss: 1.2670
Epoch 3/10


2024-03-24 19:35:21.992289: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


14/14 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.8969 - loss: 0.4205
Epoch 4/10


...

14/14 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.8941 - loss: 0.2139
Epoch 5/10


2024-03-24 19:36:58.873835: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


14/14 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.9513 - loss: 0.1331
Epoch 6/10


2024-03-24 19:37:48.305742: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


 5/14 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step - accuracy: 0.9972 - loss: 0.0795

In [ ]:
train_accuracy = model.evaluate(batched_train)
print("Training Accuracy:", train_accuracy)

valid_accuracy = model.evaluate(batched_valid)
print("Validation Accuracy:", valid_accuracy)
